In [32]:
from functools import reduce
import pandas as pd
import glob
import os
import numpy as np

# Caminho da pasta com os arquivos CSV
pasta = './'  # ajuste se necessário

# Lista todos os arquivos CSV que começam com 'investing'
arquivos_csv = glob.glob(os.path.join(pasta, '*.csv'))

# Lê e concatena todos os arquivos
dfs = [pd.read_csv(f) for f in arquivos_csv]

# Converte a coluna de datas (ajuste o nome da coluna se necessário)

# Renomeia as colunas de fechamento e mínima em cada DataFrame e faz o merge por 'Data'
dfs_renomeados = []
nomes = ['D', 'H', 'I']

for df_atual, nome in zip(dfs, nomes):
    temp = df_atual[['Data', 'Último', 'Mínima']].copy()
    temp['Data'] = pd.to_datetime(temp['Data'], dayfirst=True, errors='coerce', format='%d.%m.%Y')
    temp = temp.rename(columns={
        'Último': f'c_{nome}',
        'Mínima': f'l_{nome}'
    })
    dfs_renomeados.append(temp)

# Merge dos três DataFrames pela coluna 'Data'
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Data', how='outer'), dfs_renomeados)

# Ordena por data crescente
df = df_merged.dropna().sort_values('Data', ascending=True).reset_index(drop=True)

# Converte as colunas de preços para float (substitui vírgula por ponto)
for nome in nomes:
    df[f'c_{nome}'] = df[f'c_{nome}'].str.replace(',', '.').astype(float)
    df[f'l_{nome}'] = df[f'l_{nome}'].str.replace(',', '.').astype(float)

for nome in nomes:
    df[f'r_{nome}_taxed'] = (df[f'c_{nome}'] / df[f'c_{nome}'].shift(1) - 1) * 0.85  # considerando 15% de imposto
    df[f'ra_{nome}_bh'] = df[f'r_{nome}_taxed'].cumsum()
    df[f'r_{nome}_md'] = (df[f'l_{nome}'] / df[f'c_{nome}'].shift(1) - 1)
    df[f'stop_{nome}'] = np.where(df[f'r_{nome}_taxed'].shift(1) < 0,
                                  np.where(df[f'r_{nome}_md'].shift(1) < -0.005, df[f'r_{nome}_md'].shift(1) - 0.01, -0.01),
                                  np.where(df[f'r_{nome}_md'].shift(1) < -0.005, df[f'r_{nome}_md'].shift(1) + 0.005, -0.001)
                                  )
    #=SE(H4<I4;I4*0,85-0,0575%-0,1%;F4)
    df[f'rs_{nome}'] =np.where(df[f'r_{nome}_md'] < df[f'stop_{nome}'],
                                  df[f'stop_{nome}'] * 0.85 - 0.0575 - 0.01, df[f'r_{nome}_taxed'])
    df[f'rsa_{nome}'] = df[f'rs_{nome}'].cumsum()

df = df.dropna().sort_values('Data', ascending=True)

# Salva em Excel
df.to_excel('análise.xlsx', index=False)
df

,Data,c_D,l_D,c_H,l_H,c_I,l_I,r_D_taxed,ra_D_bh,r_D_md,...,r_H_md,stop_H,rs_H,rsa_H,r_I_taxed,ra_I_bh,r_I_md,stop_I,rs_I,rsa_I
1,2021-06-07,74.41,74.00,34.90,34.62,231.46,230.01,0.002750,0.002750,-0.002292,...,-0.023413,-0.001000,-0.068350,-0.068350,-0.000917,-0.000917,-0.007337,-0.001000,-0.068350,-0.068350
2,2021-06-08,73.85,73.83,32.40,30.16,231.47,229.64,-0.006397,-0.003647,-0.007795,...,-0.135817,-0.033413,-0.095901,-0.164251,0.000037,-0.000880,-0.007863,-0.017337,0.000037,-0.068313
3,2021-06-09,73.67,73.67,34.60,33.13,232.00,231.12,-0.002072,-0.005718,-0.002437,...,0.022531,-0.145817,0.057716,-0.106535,0.001946,0.001066,-0.001512,-0.002863,0.001946,-0.066367
4,2021-06-10,73.81,73.64,34.45,34.18,234.10,231.70,0.001615,-0.004103,-0.000407,...,-0.012139,-0.001000,-0.068350,-0.174885,0.007694,0.008760,-0.001293,-0.001000,-0.068350,-0.134717
5,2021-06-11,73.99,72.50,34.72,34.31,237.40,234.25,0.002073,-0.002030,-0.017748,...,-0.004064,-0.022139,0.006662,-0.168223,0.011982,0.020742,0.000641,-0.001000,0.011982,-0.122735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,2025-09-24,107.57,107.36,86.41,85.74,396.50,395.03,-0.000316,0.362791,-0.002323,...,0.009537,-0.024319,0.014812,-26.460124,0.003422,0.517885,0.000304,-0.027513,0.003422,-29.073966
1081,2025-09-25,106.45,106.30,83.50,82.62,397.79,393.14,-0.008850,0.353941,-0.011806,...,-0.043861,-0.001000,-0.068350,-26.528474,0.002765,0.520650,-0.008474,-0.001000,-0.068350,-29.142316
1082,2025-09-26,106.76,106.35,83.73,82.86,398.25,396.00,0.002475,0.356416,-0.000939,...,-0.007665,-0.053861,0.002341,-26.526133,0.000983,0.521633,-0.004500,-0.003474,-0.070453,-29.212769
1083,2025-09-29,107.03,107.03,86.91,84.93,398.00,396.36,0.002150,0.358566,0.002529,...,0.014332,-0.002665,0.032282,-26.493850,-0.000534,0.521099,-0.004746,-0.001000,-0.068350,-29.281119
